In [ ]:
from tkinter import *
import tkinter as tk


In [ ]:
class varType:
    def __init__(self, name, vt, default):
        self.name = name
        self.vt = vt
        self.default = default
    def __str__(self):
        return self.name + ": " + self.vt
    def __repr__(self):
        return str(self)
        
class layerType:
    def __init__(self, name, color, inCount, outCount):
        self.name = name
        self.color = color
        self.inCount = inCount
        self.outCount = outCount
        self.vars = []
    def add_var(self, name, vt, default):
        self.vars.append(varType(name, vt, default))        

In [ ]:
layers = []
Input = layerType("Input", "snow", 1, 1)
Input.add_var("Image Size X", "int", 256)
Input.add_var("Image Size Y", "int", 256)
Input.add_var("Channel", "int", 3)
Conv_2D = layerType("Conv 2D", "blue", 1, 1)
Conv_2D.add_var("Filter Size", "int", 16)
Conv_2D.add_var("Filter Count", "int", 1)
Conv_2D.add_var("Stride", "int", 1)
Max_Pool_2D = layerType("Max Pool 2D", "green", 1, 1)
Max_Pool_2D.add_var("Filter Size", "int", 16)
Max_Pool_2D.add_var("Stride", "int", 1)
Flatten = layerType("Flatten", "orange", 1, 1)
Fully_Conn = layerType("Fully Conn", "red", 1, 1)
Fully_Conn.add_var("hidden_size", "int", 64)

layers.append(Input)
layers.append(Conv_2D)
layers.append(Max_Pool_2D)
layers.append(Flatten)
layers.append(Fully_Conn)

In [ ]:
widget_height = 40

class layer:
    def __init__(self, x, y, canvas, layertype):
        self.x = x
        self.y = y
        self.layer = layertype
        self.w = self.create_oval(canvas)
        self.prev = []
        self.layervars = []
        for varT in layertype.vars:
            self.layervars.append(layerVar(varT))
    def create_oval(self, canvas):
        x1,y1 = (self.x-25), (self.y-25)
        x2,y2 = (self.x+25), (self.y+25)                              
        return canvas.create_oval(x1, y1, x2, y2, fill=self.layer.color)
    def move(self, x, y, canvas):
        self.x = x
        self.y = y
        canvas.delete(self.w)
        self.w = self.create_oval(canvas)
        if self.prev:
            for prev in self.prev:
                self.connect(prev[0], canvas, prev[1])
    def connect(self, prev, canvas, vec=None):
        if vec:
            canvas.delete(vec)
        if (prev,vec) in self.prev:
            self.prev.remove((prev,vec))
        vec = canvas.create_line(prev.x,prev.y, self.x, self.y, arrow=LAST)
        self.prev.append((prev,vec))

class layerVar:
    def __init__(self, vartype):
        self.vartype = vartype
        self.var = vartype.default
        
class currLayer:
    def __init__(self, property_frame):
        self.layer = None
        self.rect  = None
        self.property_frame = property_frame
    def select(self, node, canvas):
        self.layer = node
        if self.rect:
            canvas.delete(self.rect)
        x1,y1 = (self.layer.x-25),(self.layer.y-25)
        x2,y2 = (self.layer.x+25),(self.layer.y+25)
        self.rect = canvas.create_rectangle(x1,y1, x2,y2)
        for widget_to_destroy in self.property_frame.winfo_children():
            widget_to_destroy.destroy()
        i = 0
        for layervar in self.layer.layervars:
            tk.Label(self.property_frame, text=layervar.vartype.name).grid(row=i)
            stringv = tk.StringVar()
            stringv.set(layervar.var)
            tk.Entry(self.property_frame, textvariable=stringv).grid(row=i, column=1)
            i = i + 1
            def entry_changed_callback(*args, stringv=stringv, layervar=layervar):
                layervar.var=stringv.get()
            stringv.trace("w", callback=entry_changed_callback)


In [ ]:
canvas_width = 1280
canvas_height = 720
left_frame_width = 240
widget_height = 40

root = tk.Tk()
root.title("Draw the Network")

c = tk.Canvas(root, width=canvas_width, height=canvas_height)
c.pack(expand = YES, fill = BOTH)
message = Label( root, text = "Press and Drag the mouse to draw" )
message.pack( side = BOTTOM )

lf = tk.Frame(c, width=left_frame_width, height=canvas_height, highlightbackground="green", highlightthickness=1, bg="lavender")
c.create_window(0, 0, anchor=tk.NW, window=lf)

rf = tk.Canvas(c, width=1000,height=canvas_height)

c.create_window(left_frame_width, 0, anchor=tk.NW, window=rf)
main_canvas = rf

buttons = []
for i in range(0, len(layers)):
    b = tk.Button(lf, text=layers[i].name, width=10, anchor=tk.NW)
    b.grid(row=i, columnspan=2, sticky=tk.W)
    buttons.append(b)

lf_properties = tk.Frame(lf, bg="pale green")
lf_properties.grid(row=len(layers))

def setin(i):
    inp = layers[i]
    for button in buttons:
        button.config(highlightbackground="white")
    buttons[i].config(highlightbackground="red")

    return setInput(inp)

for i in range(0, len(layers)):
    buttons[i].config(command=lambda i=i: setin(i))

def setInput(inp):
    global layertype
    layertype = inp

list_nodes = []
currNode = currLayer(lf_properties)
layertype = None

In [ ]:
def selectORcreate( event ):
    global layertype
    if layertype is None:
        return
    x = event.x
    y = event.y
    selected = None
    for node in list_nodes:
        if (x - node.x)**2 + (y - node.y)**2 < 25**2:
            selected = node
            break
    if selected:
        currNode.select(selected, main_canvas)
    else:
        selected = layer(x, y, main_canvas, layertype)
        currNode.select(selected, main_canvas)
        list_nodes.append(selected)
        
def move( event ):
    selected = currNode.layer
    if selected is None:
        return
    selected.move( event.x, event.y, main_canvas)
    for node in list_nodes:
        for prev in node.prev:
            if prev[0] == selected:
                node.move(node.x, node.y, main_canvas)
    currNode.select(selected, main_canvas)
    
def connect( event ):
    x = event.x
    y = event.y
    selected = None
    for node in list_nodes:
        if (x - node.x)**2 + (y - node.y)**2 < 25**2:
            selected = node
            break
    if selected:
        selected.connect(currNode.layer, main_canvas)
        currNode.select(selected, main_canvas)



In [ ]:
main_canvas.bind( "<Button-1>", selectORcreate)
main_canvas.bind( "<Button-3>", connect)
main_canvas.bind( "<B1-Motion>", move)

In [ ]:
mainloop()